# Workflows Deep Dive

> "A Quote by Someone." ~ Some O Ne

Welcome to PyData and to the workshop "Workflows Deep Dive: From Data Engineering to Machine Learning."

Today we will learn about different data workflows and about how to create one for ourselves. In particular, we'll go through the following series of steps to create a full blown data project.

## Table of Contents

1. Motivation
2. What Are Workflows?
3. Our Project for Today
4. Setting Up a Monorepo
5. Data Engineering
6. Analytics
7. Data Science
8. A Bit of MLOps
    - Feature Store
    - Experiment Tracking
    - CI/CD
9. GitOps
10. Testing
11. Beyond Workflows

## 1. Motivation

## 2. What Are Workflows?

## 3. Our Project for Today

## 4. Setting Up a Monorepo

What is a Monorepo?

A "Mono" (single) "Repo" (repository) is a software development strategy where the code for multiple interrelated projects is stored in single repository.

While this strategy is common practice in the software development, it can be considered a relatively new one in the data profession.

Before we talk about why would we want to use one, it is important to highlight that a `monorepo != monolith` application. While the former can be used to build a monolithic application, the latter can still be using different repositories.

Why use one?
- You want 

Why not use one?

Let's get started?

## 5. Data Engineering

Data Engineering as a discipline is the backbone of any data endeavor undertaken by small to large organization. It is the discipline in charge of creating the data flows and infrastructure of organizations so that everyone can take advantage of data.

In this scenario, taking advantage of data might include creating data lakes, warehouses, and pipelines that move data between the former two and the processes and applications or systems that produce them. 

While it is totally possible to extract value from data without having any data engineering capabilities, having at team of these players can improve the data capabilities of any company by orders of magnitude. In fact, the value provided by data engineers can be seen through the rise of job ads for such role on LinkedIn, Indeed, Seek, and many others.

That said, engineering data systems requires skills and tools, and since I know you have the former under your belt, lets talk about the latter. For our project, we will be using [Dagster](), a data orchestration tool built for data and machine learning engineers.

Dagster has a few inportant concepts to keep in mind

## 6. Analytics

## 7. Data Science

## 8. A Bit of MLOps

## 8.1 Feature Store

## 8.2 Experiment Tracking

## 9. GitOps

## 10. Testing

## 11. Beyond Workflows